In [1]:
import json
import requests
import pickle
import threading

In [8]:
domains = ["buzzfolder.com", "examequip.com", "gradepack.com", "quizlookup.com", "wikicram.com"]
def grab_site(site):
    all_post_lists = []
    page = 1
    print(site)
    while True:
        if(page%100==0):
            print(site, page)
        response = requests.get(url="https://" + site + "/wp-json/wp/v2/posts", params={"page":page, "per_page": 100})
        # the "page > 200" part limits it to more recent questions, which are more likely to be "hot"
        # remove it to get all questions (this may take a _long_ time)
        # TODO: prevent fetching posts we already have
        if(response.status_code!=200 or page > 1000):
            break
        all_post_lists.append(response.json())
        page+=1
    pickle.dump(all_post_lists, open("v2/"+site+".raw_posts", 'wb'))
    print("data written")
threads = []
for i in range(len(domains)):
    t = threading.Thread(target=grab_site, args=(domains[i], ))
    t.start()
    threads.append(t)
for t in threads:
    # Wait for all threads to finish
    t.join()

buzzfolder.com
examequip.com
gradepack.com
quizlookup.com
wikicram.com
buzzfolder.com 100
examequip.com 100
quizlookup.com 100
wikicram.com 100
gradepack.com 100
buzzfolder.com 200
examequip.com 200
quizlookup.com 200
wikicram.com 200
gradepack.com 200
buzzfolder.com 300
examequip.com 300
quizlookup.com 300
wikicram.com 300
gradepack.com 300
buzzfolder.com 400
examequip.com 400
quizlookup.com 400
buzzfolder.com 500
wikicram.com 400
gradepack.com 400
examequip.com 500
quizlookup.com 500
buzzfolder.com 600
wikicram.com 500
examequip.com 600
gradepack.com 500
quizlookup.com 600
buzzfolder.com 700
examequip.com 700
wikicram.com 600
quizlookup.com 700
gradepack.com 600
buzzfolder.com 800
examequip.com 800
quizlookup.com 800
wikicram.com 700
gradepack.com 700
buzzfolder.com 900
examequip.com 900
quizlookup.com 900
gradepack.com 800
wikicram.com 800
buzzfolder.com 1000
data written
examequip.com 1000
data written
quizlookup.com 1000
data written
gradepack.com 900
wikicram.com 900
gradepack.co

In [9]:
# multi site mode
all_post_lists = []
for site in domains:
    this_site_file = open("v2/"+site+".raw_posts", 'rb')
    site_lists = pickle.load(this_site_file)
    this_site_file.close()
    all_post_lists.append(site_lists)
print(len(all_post_lists))

5


In [10]:
all_post_lists[0][0][0]['questions'][0]

[{'blog-post-questions-question': 'The l&acy;rger, weight-be&acy;ring b&ocy;ne &ocy;f the l&ocy;wer leg is the',
  'blog-post-questions-answer': '',
  'blog-post-questions-anchor-link': 'the-larger-weight-bearing-bone-of-the-lower-leg-is-the'}]

In [11]:
all_questions = set()
for site in all_post_lists:
    for post_list in site:
        for post in post_list:
            for question in post['questions'][0]:
    #             print(question['blog-post-questions-question'])
                all_questions.add(question['blog-post-questions-question'])
                if(question['blog-post-questions-answer']!=''):
                    print(question['blog-post-questions-answer'])

In [12]:
len(all_questions)

500648

In [13]:
import unidecode
import html

In [14]:
all_questions_formatted = set()
for q in all_questions:
    all_questions_formatted.add(unidecode.unidecode(html.unescape(q)).strip())

In [15]:
len(all_questions_formatted)

496048

In [ ]:
# search for a string
for q in all_questions_formatted:
    if "honorlock".lower() in q.lower():
        print(q)

In [19]:
questions_out = open('questions_out.txt', 'w')
for q in all_questions_formatted:
    questions_out.write(q + '\n')
questions_out.close()

In [ ]:
# split into multiple files for the question browser
!split -l 20 questions.txt questions/q_ --numeric-suffixes --additional-suffix=.txt --suffix-length=5